# Pulling Data from APIs
As a Data Analyst, there are usually two scenarios in which you will have to work with APIs:

1. You need to analyse or build reporting on your organization's data that is only available via one or multiple APIs
2. You want to enrich your organization's data with external data to improve decision making and drive business value

Therefore, being able to connect to and query from APIs is an invaluable skill and will make you more flexible when it comes to working with different data sources. We know working with APIs can be overwhelming and in order to spare you unnecessary frustration, this notebook will provide you with a structured step-by-step approach to make working with APIs as easy and fun as possible.

In this notebook we will be using the OpenWeather API https://openweathermap.org/ with the goal of enriching our existing flights data with additional weather data.

Furthermore, you're going to

1. Learn about OpenWeather API's available data and limitations
2. Sign-up to the OpenWeather API and use your API key to make your first call to the OpenWeather API
3. Learn how to adjust your API calls to get the data you need
4. Learn how to access and extract data from your JSON
5. Learn how to flatten nested JSON data and transform it into a DataFrame for future analysis
6. Learn how to make multiple calls to the API with different parameters in an automated way


## Introduction - How to use OpenWeather API

<img src="images/OpenWeather_API_Logo.jpg" width="600">

APIs all work very similar in that they use standard methods to make it easy to request and use data. Apart from their functionality, APIs can differ in multiple ways such as the type, amount, level of granularity and accuracy of the data, the authentication method, how many calls can be send per second/minute/day/in total annd the rules on using and publishing the data.

Check out their homepage and documentation (https://openweathermap.org/api) and try to answer the questions below.

* Does the API have the data I need for my use case in terms of type, completeness, granularity and accuracy?
* Is it free or paid?
* How many API calls can I make per month?
* How many API calls can I make per minute?

1. Yes
2. Mostly Free
3. 1.000.000
4. 60 only on current weather, because of limitation

### Exercises

Please go through the use cases below and answer the questions.

Use case #1: Your organization wants you to record the current weather for 100 locations every hour.

* Which API provides this data?
- Current Weather Data - API (free)
* Will you have to purchase a paid API plan?
No
* Which limition(s) could be problematic?

95 % Updatetime only in comparision to 99% // We have to query the data in 2 queries due to the 60 per min limitation.

*// Please put your answers here*

Use case #2: Your boss wants you to retrieve weather data for the last 12 months for 10 of your company's store locations to find out if certain weather conditions had an impact on sales. Your budget to get this data is 100$.

* Which API provides this data?
- 'History Bulk'
* How much will it cost you to get this data?
EUR 90 = USD 102.68
* Will the budget be sufficient?
No we need additional USD 2.68,-

*// Please put your answers here*

Use case #3: Your boss loves chocolate pudding. What she hates is rain. Unfortunately, she also hates you. One day your boss is in full rage mode, because it rained on her way back from the lunch break and she got wet. The only thing that can safe her day is her beloved chocolate pudding she put in the fridge in the morning. Unfortunately, you ate it, because as chance would have it, you hate your boss even more. Your boss orders you into her office and she is furious. She tells you the following: "If I ever get wet from rain again, I will fire you." Oh no, this is not what you had in mind. You can't get fired, you have to pay aliments for 4 children from 5 ex-wives every month. Fortunately, you are smart and have an idea: you want to write a little app that queries the weather forecast from the OpenWeather API and sends your boss a push notification whenever it is about to rain. In order to get the most recent and accurate forecast and to minimise the risk of getting fired you plan on querying the API every second. Because you're poor you want to use the free API plan.

* Which API provides the data you need?
* How accurate is the forecasted weather data?

*// Please put your answers here*

You start querying the data and after exactly 16min40sec you get the following error message:
{ "cod": 429,
"message": "Your account is temporary blocked due to exceeding of requests limitation of your subscription type. 
Please choose the proper subscription http://openweathermap.org/price"
}
* Why are you getting this error message?



*// Please put your answer here*

When looking at the data you have queried so far, you see the same data being returned multiple times. After investigating further, you find a pattern where data changes only after every 600 rows. 
* Why is that?

*// Please put your answer here*

Now that you know why you are getting the error message and why data changes only after every 600 rows, you reconsider your initial plan.
* Taking the above information into consideration, can you use the API you initially selected with a free plan to build the weather forecast app for your boss? Why / why not?

*// Please put your answers here*

In a parallel universe you decided to go with the free API plan to build your weather forecast app. Your app works as expected and your boss receives push notification warnings whenever it's about to rain. One week later you receive a letter of termination: you got fired. Reason: Your boss got wet in the rain. You are confused. You check your code and the app again. Everything works fine. You check the logs to see what happened the day your boss got wet in the rain. You find the API didn't return data for 5 minutes shortly before it started raining. You're furious and plan to sue and demand damages from the company behind OpenWeather API.
* Why will you suffer a humiliating defeat before court?

*// Please put your answer here*

## Query Data - Making calls to the API
When you type www.google.de into your web browser and press enter, your browser will send a request to the server that is hosting google's website, asking to retrieve the contents of the website so it can display it to you. Getting data from APIs works in a similar way. A request is sent to the address of the API, but instead of returning html and javascript files that will be interpreted and rendered by you browser, almost always a JSON file with data is returned. Also while you could use a web browser to send the request, it's not needed and more convenient to use a programming language like Python to do it.

In this notebook we are going to use the requests library. It is one of the most downloaded Python packages today, pulling in around 14M downloads / week. You can find out more about it here: https://pypi.org/project/requests/.
Complete the code below and import the requests package.

In [1]:
# Import requests package
import requests

Next, as mentioned above the URL of the API has to be specified. In this example we are going to pull current weather data. The documentation for this API can be found here: https://openweathermap.org/current

The url shown below is what needs to be used in order to connect to the API.

We can see that the url has 
* a fixed part: http://api.openweathermap.org/data/2.5/weather?'
* and a variable part: 'q={city name},{state code},{country_code}&appid={API key}'

<img src="images/Current_Weather_API_Call.png" width="600">

Let's take the fixed part to define a url variable.

In [2]:
# Set URL as url
url = 'http://api.openweathermap.org/data/2.5/weather?'

While the fixed part stays constant, the variable part consists of query strings or parameters, some optional some mandatory, which can be used to select or filter data or define the format or unit of measurement of the response.

Below are the optional and required parameters when pulling current weather data by city name.

<img src="images/Current_Weather_API_Call_Parameters.png" width="600">

Current weather data can be pulled not only by city name but also by

* city ID
* geographic coordinates (latitude, longitude)
* ZIP code
* cities within a rectangular zone
* cities in circle

Depending on which of the listed methods will be used, the number and names of available parameters can change. Read the documention carefully to avoid running into error messages.

In the following example we will stick with pulling the data by city name.

Let's define two variables:

1. a location variable that we will use to set the "q" parameter of the API call
2. a unit variable that we will use to set the "units" parameter

**Note: When using more than one parameter, we need to add the "&" sign, just like we would do when checking multiple conditions in an IF clause for example (p1 AND p2 AND p3).**

In [7]:
# Set location and unit variables
location = "q=Bremen"
unit = "&units=metric"

Next, we need to define a variable that includes our API key that will be used for the appid parameter. Why? Because we need to authenticate ourselves to the API when we send a request, so the API knows that we have a registered account and can notify us in case we reach our query limit.

To do this,

1. Go to you user profile and go to "My API keys" (https://home.openweathermap.org/api_keys)
2. Copy your API key and assign it to the variable api_key below 

In [8]:
# Add API key
api_key = "&appid=07713289b74ccd429ceed390d76912ad" 

Now we're almost ready to make our first API call!

You might be wondering why we split the URL into multiple parts. The answer is, it makes it easy to change our desired output in the future. Imagine you're having multiple API calls in your script and you want to change the location parameter for all of them to the same variable at the same time. All we need to do is change one line of code and all API calls will be adjusted. This is much more convenient and less error prone compared to having to find and adjust each individual API call. This approach will also be helpful when we want to make multiple calls for f.e. different locations in one code block using loops, which we will be doing later.

The last step before we can finally query the API is to combine the multiple parts into one final URL. To do this, we can simply concatenate all strings into one. Check out the code below.

In [9]:
# Combine url parts to final url: url_f
url_f = url + location + unit + api_key

# Print url_f
print(url_f)

http://api.openweathermap.org/data/2.5/weather?q=Bremen&units=metric&appid=07713289b74ccd429ceed390d76912ad


Now that we have the url, we can finally send our first API request. Since we want to get data, we need to send a GET request. This is done using the get() function from the requests function. The get() function will send a request and retrieve the response for us.

In [11]:
# Send get request
r = requests.get(url_f)

# Print r
print(r)

<Response [200]>


Weirdly, when we print the response, all we see is a code 200 message and no data. 

Did we do something wrong? No! 

Code 200 actually means that our request was successfull. Check out https://en.wikipedia.org/wiki/List_of_HTTP_status_codes to see a list of reponse codes and their definition. Knowing what the different response codes mean will help you understand error messages and resolve issues faster in the future.

So where is the data? The response that the API sent back is actually an object containing multiple elements such as cookies, encoding, headers and content. Run the code blocks below and check out the different elements of the response.

In [12]:
# Print headers
r.headers

{'Server': 'openresty', 'Date': 'Tue, 18 Jan 2022 09:21:36 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '516', 'Connection': 'keep-alive', 'X-Cache-Key': '/data/2.5/weather?q=bremen&units=metric', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST'}

In [15]:
# Print cookies
r.cookies

<RequestsCookieJar[]>

In [16]:
# Print encoding
r.encoding

'utf-8'

In [17]:
# Print content
r.content

b'{"coord":{"lon":8.8078,"lat":53.0752},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"base":"stations","main":{"temp":6.24,"feels_like":4.2,"temp_min":5.74,"temp_max":6.71,"pressure":1036,"humidity":95,"sea_level":1036,"grnd_level":1034},"visibility":10000,"wind":{"speed":2.72,"deg":264,"gust":5.53},"clouds":{"all":25},"dt":1642497407,"sys":{"type":2,"id":2016392,"country":"DE","sunrise":1642490874,"sunset":1642520517},"timezone":3600,"id":2944388,"name":"Bremen","cod":200}'

Found it! The content body holds the data. Since by default it's in JSON format, we can use the built-in JSON decoder to increase readability and prepare the data for future data manipulation.

In [19]:
# Apply JSON decoder and save in weather_hh
weather_hb = r.json()

# Print weather_hh
print(weather_hb)

{'coord': {'lon': 8.8078, 'lat': 53.0752}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 6.24, 'feels_like': 4.2, 'temp_min': 5.74, 'temp_max': 6.71, 'pressure': 1036, 'humidity': 95, 'sea_level': 1036, 'grnd_level': 1034}, 'visibility': 10000, 'wind': {'speed': 2.72, 'deg': 264, 'gust': 5.53}, 'clouds': {'all': 25}, 'dt': 1642497407, 'sys': {'type': 2, 'id': 2016392, 'country': 'DE', 'sunrise': 1642490874, 'sunset': 1642520517}, 'timezone': 3600, 'id': 2944388, 'name': 'Bremen', 'cod': 200}


This looks good already, but we can make it look even better! To do this, we are going to use the json Python library and its functions to transform the raw data into a json object and print it with proper indentation.

In [21]:
# Import json package

import json
# Create json object: json_object
json_object = json.loads(r.content)

# Print json_object
json.dumps(json_object, indent = 3)

'{\n   "coord": {\n      "lon": 8.8078,\n      "lat": 53.0752\n   },\n   "weather": [\n      {\n         "id": 802,\n         "main": "Clouds",\n         "description": "scattered clouds",\n         "icon": "03d"\n      }\n   ],\n   "base": "stations",\n   "main": {\n      "temp": 6.24,\n      "feels_like": 4.2,\n      "temp_min": 5.74,\n      "temp_max": 6.71,\n      "pressure": 1036,\n      "humidity": 95,\n      "sea_level": 1036,\n      "grnd_level": 1034\n   },\n   "visibility": 10000,\n   "wind": {\n      "speed": 2.72,\n      "deg": 264,\n      "gust": 5.53\n   },\n   "clouds": {\n      "all": 25\n   },\n   "dt": 1642497407,\n   "sys": {\n      "type": 2,\n      "id": 2016392,\n      "country": "DE",\n      "sunrise": 1642490874,\n      "sunset": 1642520517\n   },\n   "timezone": 3600,\n   "id": 2944388,\n   "name": "Bremen",\n   "cod": 200\n}'

Awesome! Now it's super easy to identify the different key-value pairs and assess the available information.

You might be saying: "Yeah, this is cool and all, but I'm not a weather expert, what do all these fields mean?" 

-> **Read the documention!**

Go to https://openweathermap.org/current#current_JSON and you will find a list of all fields and their definition that can be found in our JSON response.

As mentioned above, we can see the different key-value pairs inside the JSON output. You've probably heard the term key-value pairs before. Do you remember where? Dictionaries, exactly! If you need a quick refresh on what dictionaries are and what they do, check out the official Python documentation here: https://docs.Python.org/3/tutorial/datastructures.html#dictionaries.

Printing the type of our weather_hh variable, which is the API response decoded by the JSON decoder, we get the confirmation that we are working with a dictionary.

In [22]:
# Print type of weather_hh
type(weather_hb)


dict

We can use different techniques to access all or only specific parts of the data. One way is to loop through the dictionary and print all key, value pairs.  
Complete the code below and print all key value pairs.

In [ ]:
# Loop through and print all key-value pairs
for ___, _____ in weather_hh.items():
    print(____ + ':', ____)

Alternatively, we can print specific values by referencing their respective key.

In [37]:
# Print values in base

weather_hb['coord']
# Print values in main
weather_hb['description']

# Print values in temp
weather_hb['temp']

KeyError: 'description'

## Working with (nested) JSONs
Now that you've learned how to pull current weather data from the OpenWeather API and explore the output, it's time to convert it to a DataFrame. Having the data in a DataFrame allows us to perform data exploration, manipulation or visualization.

The DataFrame() function in the pandas package allows us to transform a dictionary into a DataFrame.
Run the code below to import the library and transform the current weather in Hamburg into a DataFrame.

In [40]:
# Import pandas package
import pandas as pd

# Transform dictionary to dataframe (will throw an error)
pd.DataFrame(weather_hb)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

Weird, we're getting an error message. Do you have an idea what could be the issue?

Don't worry if you have no clue what the problem is. Let's look at our JSON output again. Execute the code below.

In [41]:
# Print content of json_object
json.dumps(json_object, indent = 3)

'{\n   "coord": {\n      "lon": 8.8078,\n      "lat": 53.0752\n   },\n   "weather": [\n      {\n         "id": 802,\n         "main": "Clouds",\n         "description": "scattered clouds",\n         "icon": "03d"\n      }\n   ],\n   "base": "stations",\n   "main": {\n      "temp": 6.24,\n      "feels_like": 4.2,\n      "temp_min": 5.74,\n      "temp_max": 6.71,\n      "pressure": 1036,\n      "humidity": 95,\n      "sea_level": 1036,\n      "grnd_level": 1034\n   },\n   "visibility": 10000,\n   "wind": {\n      "speed": 2.72,\n      "deg": 264,\n      "gust": 5.53\n   },\n   "clouds": {\n      "all": 25\n   },\n   "dt": 1642497407,\n   "sys": {\n      "type": 2,\n      "id": 2016392,\n      "country": "DE",\n      "sunrise": 1642490874,\n      "sunset": 1642520517\n   },\n   "timezone": 3600,\n   "id": 2944388,\n   "name": "Bremen",\n   "cod": 200\n}'

We can see that some key-value pairs have further key-value pairs as values. The key "weather" for example has four key value pairs nested within: "id", "main", "description", "icon". The DataFrame() function expects dictionaries with only one level of key-value pairs, but since we have nested pairs, it throws an error.

There are multiple solutions to this problem and the one that will be the most useful to you depends on what parts of the data you ultimately need for your analysis. We need all data in our output and therefore the method we are going to use is to flatten the JSON data using the json_normalize() function from the pandas package. It flattens our JSON data and transforms it into a DataFrame. Execute the code below and check the output.

In [43]:
# Flatten json and save in weather_hh_norm
weather_hb_norm = pd.json_normalize(weather_hb, sep="_")

# Print weather_hh_norm
print(weather_hb_norm)

                                             weather      base  visibility  \
0  [{'id': 802, 'main': 'Clouds', 'description': ...  stations       10000   

           dt  timezone       id    name  cod  coord_lon  coord_lat  ...  \
0  1642497407      3600  2944388  Bremen  200     8.8078    53.0752  ...   

   main_grnd_level  wind_speed  wind_deg  wind_gust  clouds_all  sys_type  \
0             1034        2.72       264       5.53          25         2   

    sys_id  sys_country  sys_sunrise  sys_sunset  
0  2016392           DE   1642490874  1642520517  

[1 rows x 27 columns]


Awesome! Now we have a DaraFrame with 1 row and 24 columns. But wait, for some reason the "weather" column still contains data in JSON format. This is because the nested key-value pairs are in a list. Probably because sometimes the weather column can be nested further. In order to resolve this we need to use the advanced parameters of the json_normalize() function.

* record_path = specify the key that is nested deeper
* meta = specify the structure of the remaining JSON
* record_prefix = adds a prefix to the column names in the record_path to avoid duplicate column names

Execute the code below and compare the output to the output above.

In [44]:
# Advanced flattening with json_normalize()
weather_hb_df = pd.json_normalize(weather_hb, 
                                  sep="_", 
                                  record_path="weather", 
                                  meta=[["coord", "lon"], 
                                        ["coord", "lat"], 
                                        "base",
                                        ["main", "temp"],
                                        ["main", "feels_like"],
                                        ["main", "temp_min"],
                                        ["main", "temp_max"], 
                                        ["main", "pressure"], 
                                        ["main", "humidity"], 
                                        "visibility", 
                                        ["wind", "speed"], 
                                        ["wind", "deg"], 
                                        ["clouds", "all"], 
                                        "dt", 
                                        ["sys", "type"], 
                                        ["sys", "id"],
                                        ["sys", "country"],
                                        ["sys", "sunrise"],
                                        ["sys", "sunset"],
                                        "timezone",
                                        "id",
                                        "name",
                                        "cod"], 
                                  record_prefix="weather_")
# Print weather_hh_df
print(weather_hb_df)

   weather_id weather_main weather_description weather_icon coord_lon  \
0         802       Clouds    scattered clouds          03d    8.8078   

  coord_lat      base main_temp main_feels_like main_temp_min  ...  \
0   53.0752  stations      6.24             4.2          5.74  ...   

           dt sys_type   sys_id sys_country sys_sunrise  sys_sunset timezone  \
0  1642497407        2  2016392          DE  1642490874  1642520517     3600   

        id    name  cod  
0  2944388  Bremen  200  

[1 rows x 27 columns]


You're probably thinking: "Wow, this requires a lot of manual work typing all those keys in the paramaters". And yes, you're right. Unfortunately this is one of the easier methods to solve the issue. Also, it's not perfect and comes with its own limitations. If you have data that is nested even deeper you often have no other choice but to write your own flattening function. Fortunately there are other Python specialists out there who have done so and shared their work here: 

* https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
* https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame
* https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e


Alright, now that the data is in the right format, we don't have to worry about nested JSONs anymore. To make our data more interesting, since right now we only have one row, let's pull current weather data from more cities and create a proper table with multiple rows. In order to do that, we're going to combine, extend and apply everything we've learnt so far:

* Define the parameters in the request URL
* Send a request and retrieve the response using the get() function
* Decode the response using the JSON decoder .json()
* Flatten the JSON file and transform it into a DataFrame

The last step will be to combine all dataframes into one final table.

This time we want to get the current weather data of multiple locations. Unfortunately the location parameter "q" only takes one location at a time. Using a for-loop we should be able to make multiple API calls while iterating through several locations.

First, let's collect all the pieces we need below and

1. Change the location variable to locations and have to include a list with multiple locations
2. Define an empty DataFrame variable
3. Add a for-loop that iterates through all locations, flattens the JSON output and appends it to the DataFrame

**IMPORTANT: Don't run the code below too often per minute, since you only have 60 API calls per minute you can quickly run into a temporary query limit!**

In [48]:
# Set the url
url = 'http://api.openweathermap.org/data/2.5/weather?'

# Set locations
locations = ["q=Hamburg", 
             "q=Berlin", 
             "q=London", 
             "q=Madrid",
             "q=New York",
             "q=Moscow",
             "q=New York",
             "q=Ankara",
             "q=Baghdad",
             "q=Kabul",
             "q=Tokio",
             "q=Taipei",
             "q=Manila",
             "q=Auckland",
             "q=Seoul"]

# Set unit of measurement
unit = "&units=metric"

# Set API key (defined already)
api_key

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])

# Loop through all locations
for location in locations:
    # Create final url
    url_f = url + location + unit + api_key
    
    # Request data from url
    r = requests.get(url_f)
    
    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = r.json()

    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="weather", 
                                        meta=[["coord", "lon"], 
                                              ["coord", "lat"], 
                                              "base",
                                              ["main", "temp"],
                                              ["main", "feels_like"],
                                              ["main", "temp_min"],
                                              ["main", "temp_max"], 
                                              ["main", "pressure"], 
                                              ["main", "humidity"], 
                                              "visibility", 
                                              ["wind", "speed"], 
                                              ["wind", "deg"], 
                                              ["clouds", "all"], 
                                              "dt", 
                                              ["sys", "type"], 
                                              ["sys", "id"],
                                              ["sys", "country"],
                                              ["sys", "sunrise"],
                                              ["sys", "sunset"],
                                              "timezone",
                                              "id",
                                              "name",
                                              "cod"], 
                                        record_prefix="weather_",
                                        errors='ignore')
    
    # Append data to dataframe
    weather_df = weather_df.append(weather_temp_df, ignore_index=True)

# Print final dataset weather_df
weather_df

,weather_id,weather_main,weather_description,weather_icon,coord_lon,coord_lat,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,803,Clouds,broken clouds,04d,10,53.55,stations,6.25,4.33,5.53,...,1642499738,1,1263,DE,1642490720,1642520098,3600,2911298,Hamburg,200
1,803,Clouds,broken clouds,04d,13.4105,52.5244,stations,3.28,0.02,1.72,...,1642499454,2,2011538,DE,1642489622,1642519559,3600,2950159,Berlin,200
2,800,Clear,clear sky,01d,-0.1257,51.5085,stations,2.64,2.64,-1.94,...,1642499726,2,2019646,GB,1642492606,1642523074,0,2643743,London,200
3,800,Clear,clear sky,01d,-3.7026,40.4165,stations,4.53,3.64,-0.47,...,1642499953,2,2007545,ES,1642491272,1642526125,3600,3117735,Madrid,200
4,804,Clouds,overcast clouds,04n,-74.006,40.7143,stations,-0.72,-0.72,-1.98,...,1642500112,2,2039034,US,1642508186,1642542964,-18000,5128581,New York,200
5,601,Snow,snow,13d,37.6156,55.7522,stations,-4.96,-11.96,-5.86,...,1642499864,2,47754,RU,1642484760,1642512799,10800,524901,Moscow,200
6,804,Clouds,overcast clouds,04n,-74.006,40.7143,stations,-0.72,-0.72,-1.98,...,1642500112,2,2039034,US,1642508186,1642542964,-18000,5128581,New York,200
7,600,Snow,light snow,13d,32.8543,39.9199,stations,0.51,-4.73,0.51,...,1642499939,1,6947,TR,1642482423,1642517422,10800,323786,Ankara,200
8,800,Clear,clear sky,01d,44.4009,33.3406,stations,10.59,8.15,10.59,...,1642500003,NaN,NaN,IQ,1642478720,1642515582,10800,98182,Baghdad,200
9,803,Clouds,broken clouds,04d,69.4167,34.5,stations,2.16,0.11,2.16,...,1642500095,NaN,NaN,AF,1642472870,1642509421,16200,1138957,Kabul,200


Fantastic! Now we have a DataFrame with current weather information for 15 locations. What's even better, we didn't have to write a lot of complicated code. How cool is that!?  

If we wanted to, we could now create a new weather table in our database and write the weather data into it. Guess what, this is exactly what you're going to do in the upcoming project. So take a break and then move on to the next challenge, good luck!